In [1]:

from jupyter_plotly_dash import JupyterDash

import json

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash import Dash, callback_context

from dash.dependencies import Input, Output, State
import base64

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps


from project1 import AnimalShelter #python CRUD file

###########################
# Data Manipulation / Model
###########################

shelter = AnimalShelter() #sets variable to use CRUD functions

df = pd.DataFrame.from_records(shelter.readAll({})) # sets dataframe from Read in CRUD file

data2 = df # stores extra copy for later use

df['id'] = df.index 

# preset lists to allow for easy data querys in button callback

LabRMix  = pd.DataFrame(list(shelter.readAll({"breed": "Labrador Retriever Mix"})))
ChesBayR = pd.DataFrame(list(shelter.readAll({"breed": "Chesapeake Bay Retriever"})))
NewFouL  = pd.DataFrame(list(shelter.readAll({"breed": "Newfoundland"})))
GerShep  = pd.DataFrame(list(shelter.readAll({"breed": "German Shepherd"})))
AlaskMal = pd.DataFrame(list(shelter.readAll({"breed": "Alaskan Malamute"})))
OldEnShe = pd.DataFrame(list(shelter.readAll({"breed": "Old English Sheepdog"})))
SibHusk  = pd.DataFrame(list(shelter.readAll({"breed": "Siberian Husky"})))
Rotti    = pd.DataFrame(list(shelter.readAll({"breed": "Rottweiler"})))
DobPin   = pd.DataFrame(list(shelter.readAll({"breed": "Doberman Pinscher"})))
GoldR    = pd.DataFrame(list(shelter.readAll({"breed": "Golden Retriever"})))
BloodHou = pd.DataFrame(list(shelter.readAll({"breed": "Bloodhound"})))

#variables used for button controls
Water = 0
Mountain = 0
Disaster = 0
Reset = 0

#########################
# Dashboard Layout / View
#########################


app = JupyterDash('SimpleExample')

#logo for file
image_filename = 'GraziosoSalvareLogo.png'  
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# structure the screen 
app.layout = html.Div([
   
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H2('Dakota Disher'))),
    html.Hr(),
    html.Center(html.Img(id='customer-image', src='data:image/png;base64,{}'
                         .format(encoded_image.decode()), alt='customer image')),
    
    #setup buttons
    html.Div(
        children=[
            html.Button(id='WaterRescueButton', n_clicks=0, children = ['Water Rescue']),
            html.Button(id='MountainRescueButton', n_clicks=0, children = ['Mountain Rescue']),
            html.Button(id='DisasterRescueButton', n_clicks=0, children = ['Disaster Rescue']),
            html.Button(id='ResetButton', n_clicks=0, children = ['Reset']),
            
        ]),
    

    #data table settings 
    html.Hr(),
    dt.DataTable(
        id='datatable-id',

        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": False} for i in df.columns if i != "id"
        ],

        data=df.to_dict('records'), 
        sort_action='native',
        sort_by=[{'column_id': '1', 'direction': 'asc'}],
        sort_mode="single",
        column_selectable= False,
        row_selectable="single",
        selected_columns=['1'],
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10,        
    ),

    html.Br(),
    html.Hr(),
    html.Div(html.Footer('Dakota Disher SNHU CS-340')),

    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
     html.Div(
         className='row',              
         style={'display' : 'flex'},
          children=[
              html.Div(
                  id='graph-id',
                  className='col s12 m6',
                  ),
              html.Div(
                  id='map-id',
                  className='col s12 m6',
                  ),
     ]),
    html.Div(html.Footer('Dakota Disher SNHU CS-340')),
])

#############################################
# Interaction Between Components / Controller
#############################################

#button callback
@app.callback(
    Output('datatable-id', "data"),       
    
    [Input('WaterRescueButton', 'n_clicks'), Input('MountainRescueButton', 'n_clicks'),
     Input('DisasterRescueButton', 'n_clicks'), Input('ResetButton', 'n_clicks')
     
     ]
)

def on_click(WaterButton, MountainButton, DisasterButton, ResetButton):    
    
    
    global Water
    global Mountain
    global Disaster
    global Reset   
    global data2
    global data1
   
    # compares values of button n_clicks to preset variables if clicks are more if statement executes
    # then variable is updated.
                                              
    if (int(WaterButton) > Water):    
        
        frames = [LabRMix,ChesBayR,NewFouL] # takes earlier preset search lists and combines results
        df = pd.concat(frames)              # to make new dataframes with desired breeds
        Water = Water + 1 
        col_name = "sex_upon_outcome"  #sets variables to filter sex 
        filter_value = "Intact Female"        
        df = df.loc[df[col_name].str.contains(filter_value)] #filters entries from dataframes by sex
        col_name2 = "age_upon_outcome_in_weeks"
        filter_value2 = 26
        operator = "ge"        
        df = df.loc[getattr(df[col_name2], operator)(filter_value2)]# filters entries by bottom age
        filter_value3 = 156
        operator2 = "le"
        df = df.loc[getattr(df[col_name2], operator2)(filter_value3)]# filters entries by top age
        df["id"]=df.index   
        data1 = df
       
                         #### steps repeated for each button ####
        
            
    if (int(MountainButton) > Mountain):
        
        frames = [GerShep,AlaskMal,OldEnShe,SibHusk,Rotti]
        df = pd.concat(frames)
        Mountain = Mountain + 1 
        col_name = "sex_upon_outcome"
        filter_value = "Intact Male"        
        df = df.loc[df[col_name].str.contains(filter_value)]
        col_name2 = "age_upon_outcome_in_weeks"
        filter_value2 = 26
        operator = "ge"        
        df = df.loc[getattr(df[col_name2], operator)(filter_value2)]
        filter_value3 = 156
        operator2 = "le"
        df = df.loc[getattr(df[col_name2], operator2)(filter_value3)]
        df["id"]=df.index
        data1=df
        
    if (int(DisasterButton) > Disaster):
        frames = [DobPin,GerShep,GoldR,BloodHou,Rotti]
        df = pd.concat(frames)
        Disaster = Disaster + 1
        col_name = "sex_upon_outcome"
        filter_value = "Intact Male"        
        df = df.loc[df[col_name].str.contains(filter_value)]
        col_name2 = "age_upon_outcome_in_weeks"
        filter_value2 = 20
        operator = "ge"        
        df = df.loc[getattr(df[col_name2], operator)(filter_value2)]
        filter_value3 = 300
        operator2 = "le"
        df = df.loc[getattr(df[col_name2], operator2)(filter_value3)]        
        df["id"]=df.index
        data1=df
        
    if (int(ResetButton) > Reset):  #reset button loads orignial stored data 
        df = data2
       
        Reset = Reset + 1
        data1=df
    
    
    
    
    return df.to_dict('records') #send updated data back to be displayed 


# highlights rows to see which row is currently selected

@app.callback(Output('datatable-id', 'style_data_conditional'),
              [Input('datatable-id', 'derived_virtual_selected_row_ids')]
              
              )

def update_row_style(selRows):
    
  
    return [
         {"if": {"filter_query": "{{id}} ={}".format(i)},"backgroundColor": "#D2F3FF", }
        for i in selRows
    ]

# can not get to graph to work with data in AAC will work with predefined data stored in PX

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
    Input('datatable-id', 'derived_virtual_selected_row_ids')])

def update_graph(viewData, rows):
    global data1
    
    #dataset = pd.DataFrame.from_dict('records')
    
    #valueslab = dataset['breed'].value_counts()['Labrador Retriever Mix']
    #valuesCBR = dataset['breed'].value_counts()['Chesapeake Bay Retriever']
    #valuesNFL = dataset['breed'].value_counts()['Newfoundland']
    #values = dataset['breed'].value_counts()
    
    
    dff= px.data.iris()
    
    #fig = px.pie(dataset, values = dataset.value_counts.values, names = dataset.value_counts.index)
    fig = px.scatter(dff, x="sepal_width", y="sepal_length")
    #fig = px.pie(dataset, values="breed", names="breed")
    
        
    return [ dcc.Graph(figure=fig ) ]

# map callback

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
    Input('datatable-id', 'derived_virtual_selected_row_ids')])
def update_map(viewData, rows):    
    
    dff = pd.DataFrame.from_dict(viewData)        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48],
               zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                # Marker with tool tip and popup
                dl.Marker(position=[int(dff.iloc[rows, 13]),int(dff.iloc[rows, 14])], children=[
                    dl.Tooltip(dff.iloc[rows, 4]), 
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(dff.iloc[rows, 9])
                    ])    
                ])       # data in selected row is taken from file and used to update marker
            ])           # marker location is stuck at first loaded data
        ]

app # allows program to run